In [59]:
from PIL import Image
import torch
from torch import Tensor, linalg
import numpy as np
from IPython.display import Latex
import matplotlib.pyplot as plt

In [60]:
torch.set_default_dtype(torch.float64)

image_count_per_person = 10
person_count = 40
images = [Image.open(f"../data/s{j + 1}/{i + 1}.pgm") for j in range(person_count) for i in range(image_count_per_person)]

#A 3d array
all_data = np.array(images)

#A 2d array (n*d) => (400*10304)
all_data.resize((image_count_per_person * person_count, images[0].width * images[0].height))

all_data = Tensor(all_data)

labels = torch.tensor([i for i in range(person_count) for _ in range(image_count_per_person)])

# Split the Dataset into Training and Test sets
Split dataset into training and test data taking the even indexed rows for testing and the odd indexed rows for training

In [61]:
odd_indices = [i for i in range(len(all_data)) if i % 2 == 1]
even_indices = [i for i in range(len(all_data)) if i % 2 == 0]

training_data = all_data[odd_indices]
test_data = all_data[even_indices]

training_labels = labels[odd_indices]
test_labels = labels[even_indices]

print(training_data.shape)


torch.Size([200, 10304])


# Classification using LDA
## Original LDA Algorithm
Running Time: $O(d^3)$ to calculate the eigen values and eigen vectors of $\Sigma_{d \times d}$ matrix\
\
![lda](../res/lda.PNG)

In [133]:
def LDA(D , y):
    n_features = len(D[0])
    n_classes = len(y)

    overall_mean = torch.mean(D ,dim=0)

    # between-class scatter
    Sb = torch.zeros((n_features , n_features))

    # within-class scatter
    S = torch.zeros((n_features , n_features))

    for i in range (n_classes):
        Kth_class = D[y == i]
        cur_mean = torch.mean(Kth_class , dim=0)
        
        # calculate between class scatter matrix
        centered_kth_mean = (cur_mean - overall_mean).unsqueeze(1)
        
        Sb += Kth_class.shape[0] * Tensor.matmul(centered_kth_mean,centered_kth_mean.T)
        print(Sb)
        # calculate within class scatter matrix
        centered_kth_class = Kth_class - cur_mean
        S += Tensor.matmul(centered_kth_class.T,centered_kth_class)
    
    A = linalg.inv(S) @ Sb
    eigenvalues, eigenvectors = linalg.eig(A)
    idxs = torch.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:,idxs]

    return eigenvectors[0:n_classes-1]

In [134]:
projection_matrix = LDA(training_data,training_labels)   #(10304 * 39)

projected_training_matrix = training_data @ projection_matrix
projected_test_matrix = test_data @ projection_matrix


tensor([[7936.1280, 7623.3840, 8531.7360,  ..., 3073.6560, 3575.6400,
         3218.0760],
        [7623.3840, 7322.9645, 8195.5205,  ..., 2952.5305, 3434.7325,
         3091.2593],
        [8531.7360, 8195.5205, 9172.0445,  ..., 3304.3345, 3843.9925,
         3459.5933],
        ...,
        [3073.6560, 2952.5305, 3304.3345,  ..., 1190.4245, 1384.8425,
         1246.3583],
        [3575.6400, 3434.7325, 3843.9925,  ..., 1384.8425, 1611.0125,
         1449.9112],
        [3218.0760, 3091.2593, 3459.5933,  ..., 1246.3583, 1449.9112,
         1304.9201]])
tensor([[18906.0560, 19115.5780, 20155.0820,  ...,  4345.3620,  4219.6900,
          3629.0970],
        [19115.5780, 19362.2890, 20372.2410,  ...,  4284.7810,  4109.4450,
          3521.8485],
        [20155.0820, 20372.2410, 21487.7290,  ...,  4651.7890,  4526.4050,
          3895.0965],
        ...,
        [ 4345.3620,  4284.7810,  4651.7890,  ...,  1337.8490,  1459.5050,
          1294.0065],
        [ 4219.6900,  4109.4450,  4526.

KeyboardInterrupt: 